In [1]:
import sys
sys.path.append('../')
from  mongo.lib.mongo_db  import db_conn
import pandas as pd
import numpy as np
from pyecharts import Bar,configure,Pie
import time
from es_select import ES
import itertools


In [2]:
BEGIN_DATE='20181005'
END_DATE='20181006'

In [3]:
def date_to_time(date):
    timeArray = time.strptime(date, "%Y%m%d")
    time_semp = time.mktime(timeArray)
    return time_semp   # 2013
def date_to_date(date):
    timeArray = time.strptime(date, "%a %b %d %H:%M:%S %Y")
    time_semp = time.strftime("%Y%m%d%H%M%S", timeArray);
    return time_semp   # 2013

def time_to_today(timeStamp):
    timeArray = time.localtime(int(timeStamp))
    timeArray =time.strftime("%Y-%m-%d %H:%M:%S",timeArray)
    return timeArray   # 2013

## 1、查询20181005日联通出现 5017 （尊敬的客户您好，系统检测您的访问过于频繁，为保障您的账户安全请明天再试。）

In [4]:
data=db_conn.sid_info.find({"end_time":{'$gte':date_to_time(BEGIN_DATE),'$lte':date_to_time(END_DATE)},'status':5017})
data1=list(data)

In [5]:
data1_rpt_pd=pd.DataFrame(x for x in data1)

## 2、将代理信息和目标URL取出，统计请求次数、使用代理数量、代理去重列表

In [6]:
def get_proxy(data1_rpt_pd):
    all_sid_http_proxy=[]
    for sid in data1_rpt_pd.sid.values.tolist():
        es=ES(END_DATE)
        one_sid_http_proxy={}
        #print (sid)
        ret_list=es.get_all_ret(sid)
        #print(len(ret_list))
        one_sid_http_proxy['sid']=sid
        one_sid_http_proxy['tel']=data1_rpt_pd.loc[data1_rpt_pd.sid.values==sid,'tel'].values.tolist()[0]
        one_sid_http_proxy['status']=data1_rpt_pd.loc[data1_rpt_pd.sid.values==sid,'status'].values.tolist()[0]
        one_sid_http_proxy['http_proxy']=[]
        one_sid_http_proxy['url_base']=[]
        one_sid_http_proxy['dates']=[]
        for i,one_http in enumerate(ret_list):
            one_sid_http_proxy['http_proxy'].append(one_http['_source']['MSG']['proxies']['http'].replace('http://','').split('.')[0])
            one_sid_http_proxy['url_base'].append(one_http['_source']['MSG']['url'])
            one_sid_http_proxy['dates'].append(date_to_date(one_http['_source']['DATETIME']))
        l = [(k, len(list(g))) for k, g in itertools.groupby(one_sid_http_proxy['http_proxy'])]
        one_sid_http_proxy['http_req_count']=l
        one_sid_http_proxy['req_count']=len(ret_list)
        one_sid_http_proxy['http_unique']=list(set(one_sid_http_proxy['http_proxy']))
        one_sid_http_proxy['http_unique_count']=len(one_sid_http_proxy['http_unique'])
        all_sid_http_proxy.append(one_sid_http_proxy)
    return all_sid_http_proxy
        

In [7]:
all_sid_http_proxy=get_proxy(data1_rpt_pd)

In [8]:
all_sid_http_proxy_pd=pd.DataFrame(x for x in all_sid_http_proxy) 

In [9]:
all_sid_http_proxy_pd=all_sid_http_proxy_pd.fillna(0)

## 3、将数据按照手机号码组织，并获取单个客户的请求此次，使用的代理数量

In [10]:
def chain_l(l):
    ret_dict={}
    for one in l:
        if one[0] in ret_dict:
            ret_dict[one[0]]+=one[1]
        else:
            ret_dict[one[0]]=one[1]
    return ret_dict

In [11]:
import itertools
def get_new_tel_list(all_sid_http_proxy_pd):
    new_tel_list=[]
    for tel in all_sid_http_proxy_pd.tel.unique().tolist():
        new_tel_dict={}
        tmp=all_sid_http_proxy_pd.loc[all_sid_http_proxy_pd.tel.values==tel,['tel','req_count','http_req_count','http_unique_count','http_unique']]
        tmp=tmp.fillna(0)
        #print(tmp.shape)
        new_tel_dict['tel']=tel
        new_tel_dict['req_count']=tmp.req_count.values.sum()
        #print(tmp.http_req_count.values.tolist())
        l=list(itertools.chain(*tmp.http_req_count.values.tolist()))
        l_dict=chain_l(l)
        new_tel_dict['http_req_count']=tuple(l_dict.values())
        new_tel_dict['http_unique_count']=tmp.http_unique_count.values.sum()
        new_tel_dict['http_unique']= tuple(l_dict)
        new_tel_list.append(new_tel_dict)
    new_tel_pd=pd.DataFrame(x for x in new_tel_list)
    return new_tel_pd

In [12]:
new_tel_pd=get_new_tel_list(all_sid_http_proxy_pd)

## 4、当天只使用同一个代理但是被封的客户

In [13]:
filte=np.logical_and(new_tel_pd.http_unique_count.values==1,True)
tmp_1=new_tel_pd.loc[filte,['tel','req_count','http_unique_count','http_unique']]

In [14]:
tmp_1

,tel,req_count,http_unique_count,http_unique
0,18583859293,15,1,"(squidsz33,)"
1,18629867381,15,1,"(squidsz32,)"
2,13068953458,16,1,"(squidsz31,)"
3,17622997808,16,1,"(squidsz30,)"
4,17687508704,15,1,"(squidsz28,)"
5,17620464479,15,1,"(squidsz29,)"
6,18566007600,23,1,"(squidsz27,)"
7,13197773699,15,1,"(squidsz26,)"
8,13027661720,16,1,"(squidsz25,)"
9,13039438688,22,1,"(squidsz24,)"


### 将出现5017的客户按照手机号进行反查，查看当天调用量

In [15]:
data=db_conn.sid_info.find({"end_time":{'$gte':date_to_time(BEGIN_DATE),'$lte':date_to_time(END_DATE)},'tel':{'$in':tmp_1.tel.values.tolist()}})
data2=list(data)
data2=pd.DataFrame(x for x in data2)
one_sid_http_proxy=get_proxy(data2)
one_tel_http_proxy_pd=pd.DataFrame(x for x in one_sid_http_proxy) 
one_tel_http_proxy_pd=get_new_tel_list(one_tel_http_proxy_pd)

In [16]:
one_tel_http_proxy_pd.loc[:,['tel','req_count','http_unique_count','http_unique']]

,tel,req_count,http_unique_count,http_unique
0,13003096351,15,1,"(squidsz03,)"
1,13005187620,24,1,"(squidsz09,)"
2,13005786866,23,1,"(squidsz77,)"
3,13006821440,22,1,"(squidsz41,)"
4,13007575079,22,1,"(squidsz01,)"
5,13007718051,22,1,"(squidsz42,)"
6,13011025538,22,1,"(squidsz79,)"
7,13011818149,22,1,"(squidsz23,)"
8,13013395678,22,1,"(squidsz77,)"
9,13014188103,24,1,"(squidsz86,)"


### 接下来分析 代理 squidsz40，在这次请求前后是否正常

In [17]:
def sid_get_err_code(sid):
    data=db_conn.sid_info.find_one({"sid":sid,},{'_id':0,'status':1})
    return data.get('status',1)

In [18]:
def sid_get_flow_type(sid):
    data=db_conn.sid_info.find_one({"sid":sid,},{'_id':0,'tel_info':1,'status':1})
    return data['tel_info']['flow_type']

In [19]:
def anly_one_ip_proxy(one_ip_proxy):
    es=ES(END_DATE)
    proxy_ret=es.get_proxy_ret(one_ip_proxy)
    del es
    one_sid_http_proxy_list=[]
    for one in proxy_ret:
        one_sid_http_proxy={}
        one_sid_http_proxy['http_proxy']=one_ip_proxy
        one_sid_http_proxy['url_base']=(one['_source']['MSG']['url'])
        one_sid_http_proxy['dates']=(date_to_date(one['_source']['DATETIME']))
        one_sid_http_proxy['sid']=((one['_source']['MSG']['sid']))
        one_sid_http_proxy_list.append(one_sid_http_proxy)
    one_sid_http_proxy_pd=pd.DataFrame(x for x in one_sid_http_proxy_list)
    #one_sid_http_proxy_pd.groupby('sid')['http_proxy'].size().reset_index()
    
    ip_proxy_sort_pd=one_sid_http_proxy_pd.groupby('sid')['dates'].max().reset_index().sort_values(by="dates")
    ip_proxy_sort_pd_t=one_sid_http_proxy_pd.groupby('sid')['http_proxy'].size().reset_index()
    ip_proxy_sort_pd=pd.merge(ip_proxy_sort_pd,ip_proxy_sort_pd_t,on=['sid'],how='left') 
    ip_proxy_sort_pd['status']=ip_proxy_sort_pd.sid.map(sid_get_err_code)
    ip_proxy_sort_pd['flow_type']=ip_proxy_sort_pd.sid.map(sid_get_flow_type)
    print(ip_proxy_sort_pd.http_proxy.sum())
    return ip_proxy_sort_pd.loc[ip_proxy_sort_pd.flow_type.values=='10010',:]

In [20]:
anly_one_ip_proxy('http://squidsz40.dianhua.cn:8080')

4657


,sid,dates,http_proxy,status,flow_type
1,SID257981f90a3c4af8aac85049c45f963b,20181005000930,27,0,10010
5,SIDcdf638c80ff849a192e36aa5f81e65a8,20181005002941,22,0,10010
7,SID1c0f2ea3a901484b8feb0be6bcf2231f,20181005005156,45,0,10010
10,SID10f6964c394143a089dc4a0956573a84,20181005011259,24,0,10010
13,SIDda90db255d34440385d7d6356720898b,20181005014918,32,0,10010
16,SID0e27630ecf164558b9bb013a69e9516a,20181005023033,5,0,10010
19,SIDaceabd1b75ce411b9863b1dd507b1078,20181005040645,32,0,10010
22,SID1b74adcec2bc484993c4bd0285f241a2,20181005062508,28,0,10010
25,SID029aaf2221ac4778a128188ca8fee931,20181005072914,2,5017,10010
29,SID962ff903e57742e89a23927cc0128098,20181005081321,36,0,10010


In [21]:
anly_one_ip_proxy('http://squidsz08.dianhua.cn:8080')

3066


,sid,dates,http_proxy,status,flow_type
0,SIDcefc3b9ba8f4468e8a9895603166e5c2,20181005000043,2,0,10010
4,SIDd4691bf160bc49afa86af0857bacd041,20181005002141,18,0,10010
6,SID87d7062f952747b0834e7c4304adb063,20181005004018,15,0,10010
9,SID37eac4cd21704d9f81f18d41670f9438,20181005010417,31,0,10010
12,SID0d342fdefc484ae78f7c23e771cbd0eb,20181005012939,40,0,10010
14,SID0ffc93035d4b4fc6a6fb95c366e57d87,20181005020703,28,0,10010
18,SIDf59c853f0f4e4630be1a42601d761c0a,20181005032645,3,0,10010
21,SIDbdf70c0179c7470e8c676f768427c07e,20181005052500,18,0,10010
24,SID650751eef8cf46f5997db66e4bc22eb4,20181005071435,36,0,10010
27,SIDe5663c5e5f9946159ef530b23c3732bb,20181005075541,31,0,10010


In [22]:
anly_one_ip_proxy('http://squidsz09.dianhua.cn:8080')

3531


,sid,dates,http_proxy,status,flow_type
0,SID9b18585e0f8548f6879b87e545a9d5b3,20181005000159,4,3003,10010
3,SID0c2c63b5ce254d9a8ed8d9e507cfe8ea,20181005002125,2,0,10010
5,SID6053a012f6224cc7acdbfd8f65f75d8a,20181005004138,39,0,10010
7,SIDe8c19fbe08f14715a7ca4ca32104321a,20181005010344,3,5017,10010
11,SID001a49316a084c85af44692963a4386c,20181005012950,5,0,10010
13,SIDd31d07ebf99f49bb9c65bafe528d1ee2,20181005020813,32,0,10010
17,SID470827b282f44ed99f67a325905d0edc,20181005032712,17,0,10010
20,SID92566487a1a745fa9d880b4d0b854af2,20181005053514,33,0,10010
23,SIDc7de5c1baf7e4c9bbd83f1ff05b5a99f,20181005071452,24,0,10010
26,SIDfb0949982b004413a1a66a4acbaf2ed6,20181005075541,34,0,10010


In [23]:
anly_one_ip_proxy('http://squidsz32.dianhua.cn:8080')

3998


,sid,dates,http_proxy,status,flow_type
1,SID731029da7bd041379903485bdbfa3285,20181005000733,2,0,10010
5,SID12c456dc9b584f0789d217442661006b,20181005002709,38,0,10010
7,SIDe17d9ff4e4a24f3fae518881c87bce96,20181005004920,17,0,10010
10,SIDf56113d873a643e8a82795b1394fce8f,20181005011002,45,0,10010
13,SID0897e92d78fb480ba5f28d8eaf406e4e,20181005014521,32,0,10010
15,SID89da3c28132644b7879146ba8c9fec64,20181005022423,2,3003,10010
19,SIDf1a286a06ea14a8e8318743dbcb7f26a,20181005034653,27,0,10010
22,SIDc8e52fbe910c400f937a17b10b4d7025,20181005061440,8,3003,10010
25,SID66a2c51c9bd8419299bdf1c3e94aa959,20181005072313,32,0,10010
29,SID96f782cf54d2489091da287c3a1ca3ff,20181005081103,27,0,10010


In [24]:
anly_one_ip_proxy('http://squidsz35.dianhua.cn:8080')

3836


,sid,dates,http_proxy,status,flow_type
2,SIDa449407251c94c728d4b3ced6cf7f9d4,20181005000920,53,0,10010
6,SIDdf4551488b9545f290a7f69655d51552,20181005005042,28,0,10010
9,SIDc613d72eab334e13a64347e7945274df,20181005011038,40,0,10010
14,SIDea8291c61f5b477990ae2e6b1072e80e,20181005022758,2,4999,10010
17,SID3727da9bf5b749c194dc9f4b83a04891,20181005035028,7,0,10010
20,SIDc8e52fbe910c400f937a17b10b4d7025,20181005061646,7,3003,10010
23,SID029aaf2221ac4778a128188ca8fee931,20181005072843,3,5017,10010
27,SID348d21b2604441979634aedb82270a07,20181005081207,62,0,10010
31,SID6782260f757147dca876fb9b4e76d20a,20181005084601,23,0,10010
35,SID7159b301876249a9b6fee9900c3122d2,20181005091354,36,0,10010


#### 出现了请求频繁，该代理之后的其他会话中还是可以正常使用，并爬取成功，有的代理在后续的会话中不再成功。

## 5、使用多个代理被封情况分析
req_count 为请求官网次数

http_req_count 为http_unique中代理对应请求次数

http_unique_count 为使用代理个数

http_unique 为被使用过的代理

### A、使用代理数量超过1个的客户

In [25]:
filte=np.logical_and(new_tel_pd.http_unique_count.values>1,True)
tmp_2=new_tel_pd.loc[filte,['tel','req_count','http_req_count','http_unique_count','http_unique']]
tmp_2

,tel,req_count,http_req_count,http_unique_count,http_unique
16,13256371703,30,"(15, 15)",2,"(squidsz10, squidsz87)"
18,18600950951,37,"(15, 22)",2,"(squidsz08, squidsz03)"
25,13036666967,30,"(15, 15)",2,"(squidsz69, squidsz90)"
27,13004960337,29,"(22, 7)",2,"(squidsz66, squidsz64)"
29,17623647465,30,"(15, 15)",2,"(squidsz62, squidsz49)"
32,16607597250,38,"(16, 22)",2,"(squidsz46, squidsz41)"
33,18602064639,37,"(15, 22)",2,"(squidsz43, squidsz89)"
34,17677563578,81,"(9, 6, 8, 2, 6, 4, 10, 6, 2, 7, 2, 7, 2, 6, 2, 2)",16,"(squidsz48, squidsz28, squidsz89, squidsz32, s..."
37,13178239059,27,"(4, 23)",2,"(squidsz74, squidsz78)"
41,15691405608,34,"(3, 2, 22, 5, 2)",5,"(squidsz25, squidsz28, squidsz71, squidsz36, s..."


### B、分析代理总调用次数小于100次的客户

In [26]:
filte=np.logical_and(new_tel_pd.http_unique_count.values>1,new_tel_pd.req_count.values<100)
tmp_3=new_tel_pd.loc[filte,['tel','req_count','http_req_count','http_unique_count','http_unique']]
tmp_3

,tel,req_count,http_req_count,http_unique_count,http_unique
16,13256371703,30,"(15, 15)",2,"(squidsz10, squidsz87)"
18,18600950951,37,"(15, 22)",2,"(squidsz08, squidsz03)"
25,13036666967,30,"(15, 15)",2,"(squidsz69, squidsz90)"
27,13004960337,29,"(22, 7)",2,"(squidsz66, squidsz64)"
29,17623647465,30,"(15, 15)",2,"(squidsz62, squidsz49)"
32,16607597250,38,"(16, 22)",2,"(squidsz46, squidsz41)"
33,18602064639,37,"(15, 22)",2,"(squidsz43, squidsz89)"
34,17677563578,81,"(9, 6, 8, 2, 6, 4, 10, 6, 2, 7, 2, 7, 2, 6, 2, 2)",16,"(squidsz48, squidsz28, squidsz89, squidsz32, s..."
37,13178239059,27,"(4, 23)",2,"(squidsz74, squidsz78)"
41,15691405608,34,"(3, 2, 22, 5, 2)",5,"(squidsz25, squidsz28, squidsz71, squidsz36, s..."


#### 查看这些客户当日是否有其他返回码的请求

In [27]:
data=db_conn.sid_info.find({"end_time":{'$gte':date_to_time(BEGIN_DATE),'$lte':date_to_time(END_DATE)},'tel':{'$in':tmp_3.tel.values.tolist()}})
data2=list(data)
data2=pd.DataFrame(x for x in data2)
some_sid_http_proxy=get_proxy(data2)
some_sid_http_proxy_pd=pd.DataFrame(x for x in some_sid_http_proxy) 
some_sid_http_proxy_pd=get_new_tel_list(some_sid_http_proxy_pd)

In [28]:
some_sid_http_proxy_pd

,http_req_count,http_unique,http_unique_count,req_count,tel
0,"(24, 15, 16)","(squidsz32, squidsz47, squidsz20)",3,55,13001350252
1,"(22, 7)","(squidsz66, squidsz64)",2,29,13004960337
2,"(22, 15)","(squidsz79, squidsz61)",2,37,13007985245
3,"(15, 2, 2)","(squidsz44, squidsz43, squidsz88)",3,19,13013093266
4,"(7, 22, 2, 35)","(squidsz15, squidsz25, squidsz20, squidsz90)",4,66,13014025246
5,"(15, 15)","(squidsz66, squidsz48)",2,30,13015040999
6,"(16, 15)","(squidsz32, squidsz73)",2,31,13021788867
7,"(38, 19, 29, 30)","(squidsz20, squidsz35, squidsz81, squidsz65)",4,116,13023623122
8,"(3, 22)","(squidsz31, squidsz45)",2,25,13023987737
9,"(15, 15)","(squidsz22, squidsz76)",2,30,13027537067


#### 这些用户当日不存在其他请求，由此推测可能是频繁变更IP地址引起封禁

### C、分析请求数量超过100的代理

In [32]:
filte=np.logical_and(new_tel_pd.http_unique_count.values>1,new_tel_pd.req_count.values>50)
tmp_4=new_tel_pd.loc[filte,['tel','req_count','http_req_count','http_unique_count','http_unique']]
tmp_4

,tel,req_count,http_req_count,http_unique_count,http_unique
34,17677563578,81,"(9, 6, 8, 2, 6, 4, 10, 6, 2, 7, 2, 7, 2, 6, 2, 2)",16,"(squidsz48, squidsz28, squidsz89, squidsz32, s..."
60,18596303032,88,"(15, 15, 4, 2, 15, 2, 2, 7, 2, 2, 22)",11,"(squidsz49, squidsz72, squidsz81, squidsz77, s..."
64,18678684568,57,"(2, 2, 7, 15, 7, 22, 2)",7,"(squidsz72, squidsz75, squidsz89, squidsz43, s..."
94,17681643139,80,"(8, 11, 8, 4, 7, 8, 6, 6, 22)",9,"(squidsz77, squidsz13, squidsz32, squidsz89, s..."
213,13118018950,52,"(30, 22)",3,"(squidsz21, squidsz70)"
214,13001350252,55,"(16, 15, 24)",3,"(squidsz20, squidsz47, squidsz32)"
263,18665099526,60,"(7, 7, 8, 7, 4, 2, 8, 7, 2, 6, 2)",11,"(squidsz61, squidsz34, squidsz38, squidsz47, s..."
310,17600934668,58,"(8, 7, 7, 8, 6, 22)",6,"(squidsz19, squidsz01, squidsz34, squidsz42, s..."
347,17638345009,54,"(17, 15, 7, 15)",4,"(squidsz11, squidsz31, squidsz72, squidsz68)"
349,15691080889,73,"(7, 7, 2, 7, 9, 7, 6, 7, 6, 6, 7, 2)",12,"(squidsz85, squidsz29, squidsz90, squidsz70, s..."


#### 使用代理数量相对B情况要少，但是均存在某一个代理使用超过100的情况，可能是因为当日请求量超过阈值或者Ip地址发送变化引起

### D、分析C中代理使用量无超过100的情况发生

#### 由此可以看出，在首次请求时已经触发封禁

## 5、结论
### 1）、单次爬虫事务中，如果短时间内请求官网超过20次，可能被封禁
### 2）、多次爬虫事务中，如果使用多个ip代理请求官网，超过2个ip将可能被封禁
### 3）、联通反爬策略为，ip总请求次数超过3000，可能被封禁


# 6、采样爬取成功的联通，分析

In [33]:
data=db_conn.sid_info.find({"end_time":{'$gte':date_to_time(BEGIN_DATE),'$lte':date_to_time(END_DATE)},'status':0,'crawler_channel':{'$exists':False},'tel_info.flow_type':'10010'}).limit(40)
data2=list(data)
data2=pd.DataFrame(x for x in data2)
some_sid_http_proxy=get_proxy(data2)
some_sid_http_proxy_pd=pd.DataFrame(x for x in some_sid_http_proxy) 
some_sid_http_proxy_pd=get_new_tel_list(some_sid_http_proxy_pd)

In [34]:
some_sid_http_proxy_pd

,http_req_count,http_unique,http_unique_count,req_count,tel
0,"(5,)","(squidsz22,)",1,5,13172462334
1,"(5, 22)","(squidsz88, squidsz79)",2,27,13058019229
2,"(13,)","(squidsz85,)",1,13,15622167727
3,"(16,)","(squidsz82,)",1,16,15686176761
4,"(5,)","(squidsz80,)",1,5,17639600305
5,"(16,)","(squidsz62,)",1,16,18657206755
6,"(17,)","(squidsz74,)",1,17,13094359757
7,"(16,)","(squidsz38,)",1,16,13093559016
8,"(15,)","(squidsz37,)",1,15,13055990244
9,"(36,)","(squidsz24,)",1,36,13106639628


In [35]:
some_sid_http_proxy_pd.req_count.mean()

18.82051282051282

#### 20181005 当日成功的联通爬取平均使用代理次数为18